# TwitterTrends-2-FileToKafka

## Importing libraries

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.2.0`
import $ivy.`org.apache.spark::spark-sql-kafka-0-10:2.2.0`

import $ivy.$                                  

import $ivy.$                                             

In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.SparkSession

import org.apache.log4j.PropertyConfigurator

import org.apache.spark.sql.types._

import org.apache.spark.sql.streaming.Trigger

import org.apache.spark.sql.SparkSession


import org.apache.log4j.PropertyConfigurator

##  Creating Spark Session

*Note: As stated in [readme](https://github.com/rvilla87/Big-Data#some-things-to-consider), we will change the log lv to WARN.*

In [3]:
PropertyConfigurator.configure("C:/spark/conf/log4j.properties") // load spark's log4j configuration (set to WARN)

In [4]:
val spark = SparkSession.builder()
  .appName("TwitterFileToKafka")
  .master("local[*]")
  .getOrCreate()

spark: SparkSession = org.apache.spark.sql.SparkSession@4e97080c

##  [Spark Structure Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)

Streaming DataFrames can be created through the `DataStreamReader` interface returned by `SparkSession.readStream()`.

In [5]:
val kafkaSchema = new StructType().add("key", "String").add("value", "String")

kafkaSchema: StructType = StructType(StructField(key,StringType,true), StructField(value,StringType,true))

Now we can **define our streaming Dataframe** specifiying the **[source](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#input-sources)** (*File source*) and the **schema** we have just created:

In [6]:
// structured streaming with *.csv files 
val filesRS = spark
  .readStream
  .schema(kafkaSchema)
  .option("sep", ";")
  .csv("trendFiles/*.csv")

filesRS: org.apache.spark.sql.package.DataFrame = [key: string, value: string]

As we can see, the streaming DataFrame has the desired schema:

In [7]:
filesRS.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



Once you have defined the final result DataFrame/Dataset, all that is left is for you to start the streaming computation. To do that, you have to use the `DataStreamWriter` returned through `Dataset.writeStream()`. More info [here](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#starting-streaming-queries).

In this case, we will specify our `DataStreamWriter` in order to [stream the data to Kafka](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html).

In [8]:
// Save trends to Kafka's topic "tweeterTopic"
val queryfilesRS = filesRS
  .select("key", "value")
  .writeStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("topic", "tweeterTopic")
  .option("checkpointLocation", "jupyter/trendFiles/checkpoint-kafka")
  .trigger(Trigger.ProcessingTime("60 seconds"))
  .start()

queryfilesRS: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@b284ce9

Finally we have to run and wait for the streaming process until query is terminated, with `stop()` or with error.

*Note: Before executing next statement make sure you have [started Kafka Server](https://github.com/rvilla87/Big-Data#starting-kafka-server).*



In [ ]:
queryfilesRS.awaitTermination()